In [0]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.4/julia-1.4.1-linux-x86_64.tar.gz' -O julia.tar.gz
    tar -x -f julia.tar.gz -C /usr/local --strip-components 1
    rm julia.tar.gz
fi
    julia -e '\
        using Pkg                                                          ;\
        Pkg.update()                                                       ;\
        Pkg.add("CuArrays")                                                ;\
        Pkg.add("IJulia")                                                  ;\
        Pkg.add("Flux")                                                    ;\
        Pkg.add("MLDatasets")                                              ;\
        Pkg.add("Statistics")                                              ;\
        Pkg.update()                                                       ;\
        Pkg.precompile()                                                   ;\'
echo 'Done'

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Julia 深度學習：卷積神經網路模型簡介



In [21]:
using Flux                                                                                       
using Flux.Data: DataLoader                                                                       
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy                            
using MLDatasets                                                                                  
using Statistics        

train_X, train_y = SVHN2.traindata(Float32, 1:20000)                                              
test_X,  test_y  = SVHN2.testdata(Float32, 1:2000)                                                
train_y = onehotbatch(train_y, 1:10)                                                              
test_y = onehotbatch(test_y, 1:10)                                                               
batchsize = 1024                                                                                 
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)                          
test = DataLoader(test_X, test_y, batchsize=batchsize)           


model = Chain( Conv((3, 3), 3=>32, relu; pad=1), MaxPool((2,2)),
               Conv((3, 3), 32=>32, relu; pad=1), MaxPool((2,2)),
               Conv((3, 3), 32=>64, relu; pad=1), MaxPool((2,2)),
               flatten, Dense(1024, 512, relu), Dense(512, 128), Dense(128, 10),
               softmax)
               
loss(x, y) = logitcrossentropy(model(x), y)                                                      
function test_loss()                                                                             
    l = 0f0                                                                                       
    for (x, y) in test                                                                           
        l += loss(x, y)                                                                          
    end                                                                                           
    l/length(test)                                                                               
end   

evalcb() = @show(test_loss())                                                                    
epochs = 10                                                                                      
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.001), cb=throttle(evalcb, 10))      
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))                                          
println( )                                                                                        
println( "accuracy :" , accuracy(test_X, test_y) )  

┌ Info: Epoch 1
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 2.282226f0
test_loss() = 2.2573042f0
test_loss() = 2.2578504f0
test_loss() = 2.2536335f0
test_loss() = 2.2499337f0
test_loss() = 2.2461042f0


┌ Info: Epoch 2
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 2.2768579f0
test_loss() = 2.2497494f0
test_loss() = 2.2389488f0
test_loss() = 2.21972f0
test_loss() = 2.2060127f0


┌ Info: Epoch 3
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


...

test_loss() = 2.0618515f0
test_loss() = 2.037631f0
test_loss() = 2.022849f0
test_loss() = 2.0156567f0
test_loss() = 2.0185142f0


┌ Info: Epoch 9
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 2.1405363f0
test_loss() = 2.0490174f0
test_loss() = 2.024804f0
test_loss() = 2.013392f0
test_loss() = 2.0115752f0
test_loss() = 2.0036983f0


┌ Info: Epoch 10
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


test_loss() = 2.0620542f0
test_loss() = 2.0126472f0
test_loss() = 2.0036638f0
test_loss() = 2.0035686f0
test_loss() = 1.9965689f0
test_loss() = 1.9957476f0
test_loss() = 1.9912874f0

accuracy :0.4695


# 作業 033：訓練 CNN 學習門牌號碼資料集



# 訓練一個 CNN 模型來學習門牌號碼資料集。

In [0]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 讀取資料

In [0]:
train_X, train_y = SVHN2.traindata(Float32, 1:20000)
test_X,  test_y  = SVHN2.testdata(Float32, 1:2000)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [0]:
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×2000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  1  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  1  0  0  0  0  0  1  0  1  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [0]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

DataLoader((Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627

## CNN 模型

In [0]:
model = # write your model here

Chain(Conv((3, 3), 3=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(1024, 256, relu), Dense(256, 10), softmax)